# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a short definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#pip install pandas

In [5]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = { 'table': ['employees', 'departments', 'salaries', 'studies'],
    'definition': [
        'Contains information about employees including name, employee ID, and department ID.',
        'Includes details about departments such as department ID, department name, manager.',
        'Stores salary data for each year with employee ID, salary level, salary amount, and payment date.',
        'Educational studies, highiest educational level, name of the institution, certifications and asociated dates'
    ]
}

df = pd.DataFrame(data)
print(df)

         table                                         definition
0    employees  Contains information about employees including...
1  departments  Includes details about departments such as dep...
2     salaries  Stores salary data for each year with employee...
3      studies  Educational studies, highiest educational leve...


In [6]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [7]:
print(text_tables)

employees: Contains information about employees including name, employee ID, and department ID.
departments: Includes details about departments such as department ID, department name, manager.
salaries: Stores salary data for each year with employee ID, salary level, salary amount, and payment date.
studies: Educational studies, highiest educational level, name of the institution, certifications and asociated dates


In [8]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [9]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Which employees have a master's degree and work in the 'Sales' department?")

In [10]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "departments", "studies"]
}
```


In [11]:
#Creating the prompt, with the user questions and the tables definitions.
pqt2 = prompt_question_tables.format(tables=text_tables, question="Who is the manager with the largest number of employees reporting to them?")

In [12]:
print(return_OAI(pqt2))

{
    "tables": ["employees", "departments"]
}


In [13]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Which employees with a PhD earned more than $90,000 last year in the 'Research' department?")

In [14]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salaries", "studies"]
}


# **Prompt Variation No. 1**

In [15]:
prompt_question_tables_01 = """
Given the following tables and their content definitions,
###Tables
{tables}

Note:
- Use the 'departments' table when questions involve department-specific details.
- Use the 'employees' table for general employee information.
- Use the 'studies' table when questions involve education-related criteria.
- Use the 'salaries' table for financial or salary-related questions.

Example:
For the question "Which employees with a PhD earned more than $90,000 last year in the 'Research' department?",
the necessary tables are 'employees', 'salaries', 'studies', and 'departments'.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""

In [16]:
pqt4 = prompt_question_tables_01.format(tables=text_tables, question=
    "How many employees with a bachelor's degree or higher report to a manager in the 'Sales' department, and what is the average salary of these employees?"
    )

In [17]:
print(return_OAI(pqt4))

{
    "tables": ["employees", "departments", "studies", "salaries"]
}


In [19]:
pqt5 = prompt_question_tables_01.format(tables=text_tables, question=
    "Which employees received their highest educational degree in the same year they were hired, and in which department do they currently work?"
    )

In [20]:
print(return_OAI(pqt5))

{
  "tables": ["employees", "studies", "departments"]
}


# **Prompt Variation No. 2**

In [21]:
prompt_question_tables_02 = """
Given the following tables and their content definitions,
### Tables
{tables}

### Instructions for the Model:
- Prioritize relationships between tables based on user queries.
- Use the 'departments' table for department-related details or manager information.
- Use the 'studies' table for education-related criteria such as degrees or certifications.
- Use the 'salaries' table for queries involving financial information.
- When combining tables, identify shared fields like 'employee ID' or 'department ID' for correct relationships.

### Example Relationships:
- To find employees in a department: Combine 'employees' with 'departments' using 'department ID'.
- To check education and salaries: Combine 'employees', 'studies', and 'salaries' using 'employee ID'.

### User Question:
{question}

Return only the table names required to answer the query in a JSON format.
"""

In [22]:
pqt6 = prompt_question_tables_02.format(tables=text_tables, question=
    "Which employees were hired before obtaining their highest degree, and in which department do they work?"
    )

In [23]:
print(return_OAI(pqt6))

{
  "tables": ["employees", "studies"]
}


In [24]:
pqt7 = prompt_question_tables_02.format(tables=text_tables, question=
    "Who is the employee with the highest salary in the company, and how many years have they been working there based on their hiring date?"
    )

In [25]:
print(return_OAI(pqt7))

{
  "tables": ["employees", "salaries"]
}


In [26]:
pqt8 = prompt_question_tables_02.format(tables=text_tables, question=
    "Who has achieved the highest educational level and is associated with the highest total salary in their department?"
    )

In [27]:
print(return_OAI(pqt8))

{
  "tables": ["employees", "departments", "studies", "salaries"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# **Summary Report: SQL Query from Table Names**

Objective: The aim of this exercise was to test whether GPT-3.5-Turbo can determine the relevant tables for constructing SQL queries based on table names and their definitions. This involved feeding the model with structured prompts and evaluating its responses.
________________________________________
Findings:
1.	Effective Scenarios:
o	When table names and definitions were detailed and directly aligned with the user’s query, the model performed well, accurately identifying the necessary tables.
o	Clear and concise prompts, formatted with structured instructions, yielded accurate and consistent outputs.
o	In a challenging example where employees needed to be matched with their highest educational degree obtained in the same year they were hired, the model successfully identified only the relevant tables (employees, studies, and departments), demonstrating its ability to handle more complex and ambiguous queries.
o	For a more ambiguous query ("Who has achieved the highest educational level and is associated with the highest total salary in their department?"), the model inferred the correct set of tables (employees, departments, studies, salaries), showing that the optimized prompt improved its ability to handle unclear questions.
2.	Challenging Scenarios:
o	Hallucinations: In cases where table definitions were ambiguous or overlapped conceptually, GPT occasionally inferred irrelevant tables or omitted necessary ones.
o	Over-inclusion of Tables: Even when the user’s question was specific and did not require certain tables (e.g., salaries in creative variations), the model included them in the response.
o	Omission of Tables: For questions that involved filtering by department (e.g., "Which employees with a PhD earned more than $90,000 last year in the 'Research' department?"), the model failed to include the departments table. This suggests a lack of understanding of inter-table dependencies.
o	Sensitivity to Prompt Wording: Minor changes in the question phrasing led to inconsistencies, with the model sometimes overcompensating by adding unnecessary tables.
________________________________________
Key Learnings:
•	Prompt Design Matters: A well-structured prompt, including explicit formatting, clear task delineation, and examples of relationships, is critical for obtaining accurate responses.
•	Contextual Ambiguity: The model relies heavily on explicit definitions; vague or incomplete descriptions significantly impact accuracy.
•	Iterative Refinement: Experimenting with variations of the same query can help identify optimal approaches and refine results.
•	Dependency Clarity: Explicitly stating dependencies between tables in the prompt can help guide the model to better decisions.
•	Handling Ambiguity: Optimized prompts with detailed instructions improved the model’s ability to infer relationships, even when the query was unclear or incomplete.
•	Limitations of AI: While GPT is powerful in text understanding, it lacks inherent relational database knowledge, leading to occasional logical gaps.
________________________________________
Future Considerations:
•	Enhance table definitions with unambiguous and context-rich descriptions.
•	Include contextual examples in the prompt to guide the model effectively.
•	Validate model outputs by cross-referencing with actual database schemas to mitigate hallucinations.
•	Explore fine-tuning for domain-specific database tasks to improve precision and reliability.
•	Use prompts that explicitly prioritize relationships and include examples of common table connections to handle complex or ambiguous queries.
________________________________________
Conclusion: 
This exercise highlighted GPT-3.5-Turbo’s potential as a tool for SQL query assistance but underscored the importance of prompt engineering and validation processes. The optimized prompt significantly improved the model’s ability to infer relationships, handle ambiguity, and provide accurate table selections. While the model demonstrated strong capabilities, certain scenarios require human intervention or additional refinement to ensure accuracy.
